In [ ]:
import pandas as pd
import json
import requests

In [ ]:
pr_raw = pd.read_csv("https://ct20231211-staging.s3.amazonaws.com/pr.data.0.Current", sep='\t')
population_raw = pd.DataFrame(requests.get("https://ct20231211-staging.s3.amazonaws.com/us-population.json").json()['data'])

In [ ]:
population = pd.DataFrame()
for column in population_raw:
    column_data = population_raw[column]
    if population_raw[column].dtype == "object":
        column_data = column_data.str.strip()
    if column.strip() in ("Year", "ID Year", "Population") :
        column_data = pd.to_numeric(column_data)
    population[column.strip().lower().replace(' ','_')] = column_data

population.info()

In [ ]:
pr = pd.DataFrame()
for column in pr_raw:
    column_data = pr_raw[column]
    if pr_raw[column].dtype == "object":
        column_data = column_data.str.strip()
    if column.strip() in ("year", "value") :
        column_data = pd.to_numeric(column_data)
    if column.strip() not in ("footnote_codes"):
        pr[column.strip()] = column_data

pr.info()

In [ ]:
population

In [ ]:
pr

In [ ]:
population_2013_2018 = population[(population['year'] >= 2013) & (population['year'] <= 2018)]['population']
mean_population = population_2013_2018.mean()
std_population = population_2013_2018.std()
print("Mean US Population [2013-2018] = ", mean_population)
print("Standard Deviation [2013-2018] = ", std_population)

In [ ]:
pr_sum = pr.groupby(['series_id', 'year'])['value'].sum().reset_index()
pr_best_year = pr_sum.loc[pr_sum.groupby('series_id')['value'].idxmax()].reset_index()
print("Best year for each series:")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pr_best_year)

In [ ]:
pr_population = pd.merge(pr,population[['year','population']], on="year", how="inner")
pr_population

In [ ]:
pr_population[(pr_population["series_id"]=="PRS30006032") & (pr_population["period"]=="Q01")]